## Deliverable 2. Create a Customer Travel Destinations Map.

In [43]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Ushuaia,AR,-54.8000,-68.3000,60.46,67,75,12.66,broken clouds
1,1,Cuorgne,IT,45.3895,7.6499,48.42,70,29,8.39,scattered clouds
2,2,Kodiak,US,57.7900,-152.4072,36.91,48,100,5.75,overcast clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.80,77,12,17.09,few clouds
4,4,Victoria,HK,22.2855,114.1577,65.97,89,100,1.01,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
len(preferred_cities_df)

152

In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                152
City                   152
Country                152
Lat                    152
Lng                    152
Max Temp               152
Humidity               152
Cloudiness             152
Wind Speed             152
Weather Description    152
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_df.dropna()
len(preferred_cities_df)

152

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
3,Atuona,PF,78.80,few clouds,-9.8000,-139.0333,
5,Manggar,ID,81.72,overcast clouds,-2.8833,108.2667,
7,Saint-Philippe,RE,76.59,light rain,-21.3585,55.7679,
12,Kununurra,AU,84.18,scattered clouds,-15.7667,128.7333,
14,Mahebourg,MU,77.29,broken clouds,-20.4081,57.7000,
29,Rikitea,PF,79.02,few clouds,-23.1203,-134.9692,
30,Carnarvon,AU,78.87,clear sky,-24.8667,113.6333,
32,Honiara,SB,85.28,overcast clouds,-9.4333,159.9500,
33,Dingle,PH,83.75,scattered clouds,10.9995,122.6711,
38,Saint-Pierre,RE,76.68,clear sky,-21.3393,55.4781,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        continue
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
3,Atuona,PF,78.80,few clouds,-9.8000,-139.0333,Villa Enata
5,Manggar,ID,81.72,overcast clouds,-2.8833,108.2667,Guest Hotel
7,Saint-Philippe,RE,76.59,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
12,Kununurra,AU,84.18,scattered clouds,-15.7667,128.7333,Hotel Kununurra
14,Mahebourg,MU,77.29,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
29,Rikitea,PF,79.02,few clouds,-23.1203,-134.9692,People ThankYou
30,Carnarvon,AU,78.87,clear sky,-24.8667,113.6333,Hospitality Carnarvon
32,Honiara,SB,85.28,overcast clouds,-9.4333,159.9500,Heritage Park Hotel
33,Dingle,PH,83.75,scattered clouds,10.9995,122.6711,SEGAYA PROPERTY
38,Saint-Pierre,RE,76.68,clear sky,-21.3393,55.4781,Lindsey Hôtel


In [45]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace('', np.nan, regex=True)
hotel_df=hotel_df.dropna()
len(hotel_df)

146

In [46]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))